In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import utm

vancouver_path = "./Data/vancouvercrime.csv"
crime_mapping_path = "./Data/crime_type_mapping.csv"
df = pd.read_csv(vancouver_path)
df = df[df['YEAR'].isin([2015,2016,2017,2018,2019])]
crime_code_df = pd.read_csv(crime_mapping_path)
event_df = pd.read_csv('./Dimensions/dimension_event.csv')
event_df = event_df[event_df['event_city'].isin(["Vancouver"])]

In [ ]:
crime_list = crime_code_df["Vancouver"].tolist()

In [ ]:
population_df = pd.read_csv('./Data/Population for CSI.csv')
population_df = population_df[population_df['City'].isin(["Vancouver"])]
csi_weight = pd.read_csv('./Data/Crime weight for CSI.csv')
csi_crime_names = csi_weight['Denver Crime catrgory'].tolist()
csi_crime_record = []
for i in range(2015,2020):
    csi_crime_record.append([0]*len(csi_crime_names))

In [ ]:
loc_df_list=[]
crime_df_list=[]
fact_df_list=[]

In [ ]:
event_start_lst = event_df["event_start_date"].tolist()
event_end_lst = event_df["event_end_date"].tolist()

In [ ]:
for i, row in df.iterrows():
    # index = len(fact_df)
    index = len(loc_df_list)
    na = np.nan
    
    # Van id starts from 10^7, Den id starts from 10^6
    location_id = index + 10000000
    city = "Vancouver"
    location = row["HUNDRED_BLOCK"]
    neighborhood = row["NEIGHBOURHOOD"]
    lat,lon = utm.to_latlon(row['X'],row['Y'],10,'U') if row['X']>0 and row['Y']>0 else (na,na)
    
    
    
    crime_id = index + 20000000
    
    year = row['YEAR']
    month = row['MONTH']
    day = row['DAY']
    hour = row['HOUR']
    minute = row['MINUTE']

    crime_type_map = crime_code_df.iloc[crime_list.index(row['TYPE'])]
    crime_category_name = crime_type_map["corr Denver category"]
    crime_type_name = crime_type_map["corr Denver type"]
    crime_detail = crime_type_map["corr Denver detail"]
    
    str_month = str(month) if month>9 else '0'+str(month)
    str_day = str(day) if day>9 else '0'+str(day)
    date_id = int(str(year)+str_month+str_day)
    
    
    #Event
    event_id = np.nan
    #for idx_event, row_event in event_df.iterrows():
    for i in range(len(event_start_lst)):
        #the crime is in the event period
        if date_id>=event_start_lst[i] and date_id<=event_end_lst[i]:
            row_event = event_df.iloc[i]
            local_location = location.upper() if type(location)==str else "(｡･ω･)ﾉﾞ"
            event_location = row_event['event_location'].upper() if type(row_event['event_location'])==str else "ヾ(´･ω･｀)ﾉ"
            if local_location in event_location:
                event_id = row_event['event_key']
                break
            elif neighborhood == row_event['event_neighbourhood']:
                event_id = row_event['event_key']
            elif row_event['event_location_size'] == 'city' and np.isnan(event_id):
                event_id = row_event['event_key']
    
    is_night = 0 if 7 <= hour <= 18 else 1
    category_index_in_csi = csi_crime_names.index(crime_category_name)
    
    # location_df.loc[index] = [location_id,city,location,neighborhood,lat,lon,crime_rate]
    loc_df_list.append([location_id,city,location,neighborhood,lat,lon,0.0,year-2015])
    #crime_df.loc[index] = [crime_id,crime_category_name,crime_type_name,crime_detail,hour,minute,0,na,na,na,na,na,na]
    crime_df_list.append([crime_id,crime_category_name,crime_type_name,crime_detail,hour,minute,0,na,na,na,na,na,na,category_index_in_csi,year-2015])
    #fact_df.loc[index] = [location_id,crime_id,date_id,event_holiday_id,crime_type_map["is_traffic"],crime_type_map['is_fatal'],is_night]
    fact_df_list.append([location_id,crime_id,date_id,event_id,crime_type_map["is_traffic"],crime_type_map['is_fatal'],is_night])
    csi_crime_record[year-2015][category_index_in_csi]+=1


In [ ]:
# Crime rate
crime_rate_list=[0]*5
for i in range(0,5):
    crime_rate_list[i] = sum(csi_crime_record[i])/population_df.iloc[i]['Population']
# Crime Severity Index
for i in range(len(csi_crime_record)):
    for k in range(len(csi_crime_record[i])):
        csi_crime_record[i][k] = csi_crime_record[i][k]/population_df.iloc[i]['Population']*csi_weight.iloc[k]['Weight']*100

In [ ]:
for i in range(len(loc_df_list)):
    loc_df_list[i][-2]=crime_rate_list[loc_df_list[i][-1]]
    del loc_df_list[i][-1]
for j in range(len(crime_df_list)):
    crime_df_list[j][-2]=csi_crime_record[crime_df_list[j][-1]][crime_df_list[j][-2]]
    del crime_df_list[j][-1]

In [ ]:
fact_df = pd.DataFrame(fact_df_list, columns=['location_Key','crime_key', 'date_key','event_key', 'is_traffic','is_fatal','is_nighttime'],dtype="Int64")
location_df = pd.DataFrame(loc_df_list, columns=['location_Key','city','location_name', 'neighborhood', 'latitude','longitude','crime_rate_percentage'])
crime_df= pd.DataFrame(crime_df_list, columns=['crime_key','crime_category_name', 'crime_type_name','crime_details',
                                 'start_hour', 'start_minute',"start_second",
                                 'end_hour', 'end_minute',"end_second",
                                 'report_hour', 'report_minute',"report_second","crime_severity_index"])

In [ ]:
crime_df.head()

In [ ]:
location_df.head()

In [ ]:
fact_df.head()

In [ ]:
fact_df.to_csv(r'.\Dimensions\fact_vancouver.csv',index=False)
crime_df.to_csv(r'.\Dimensions\crm_vancouver.csv',index=False)
location_df.to_csv(r'.\Dimensions\loc_vancouver.csv',index=False)